In [ ]:
#|default_exp parking_tools

In [ ]:
#|export
"""
Fintraffic Parking helpers for “Arena Buddy”.
--------------------------------------------

* Uses v1 REST (2025-04-08 spec)
* No authentication needed for GET
"""

from __future__ import annotations
import math
import datetime as dt
import httpx
from typing import Dict, Tuple, List

# ── Constants ─────────────────────────────────────────────────────────────
_FT_BASE = "https://parking.fintraffic.fi/api/v1"
_HEAD    = {"Digitraffic-User": "datatalks-bot"}           # track usage
_EARTH_R = 6_371_000.0                                     # metres
AREENA   = (60.1778, 24.7858)                              # Urheilupuisto

# single async client reused by all helpers
_client = httpx.AsyncClient(headers=_HEAD, timeout=10)

# ── Geometry helpers ──────────────────────────────────────────────────────
def _haversine(a: Tuple[float, float], b: Tuple[float, float]) -> float:
    """Great-circle distance (m) between two (lat, lon) pairs."""
    lat1, lon1, lat2, lon2 = map(math.radians, (*a, *b))
    dlat, dlon = lat2 - lat1, lon2 - lon1
    h = (math.sin(dlat / 2) ** 2 +
         math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)
    return _EARTH_R * 2 * math.asin(math.sqrt(h))

def _latlon(fac: dict) -> Tuple[float, float]:
    """Return (lat, lon) from any facility/hub row (v0 or v1 schema)."""
    if "lat" in fac and "lon" in fac:          # legacy fields
        return fac["lat"], fac["lon"]

    loc = fac.get("location") or {}
    coords = loc.get("coordinates")
    if coords is None:
        raise KeyError("No coordinates in location")

    gtype = loc.get("type", "").upper()

    if gtype == "POINT":                       # [lon, lat]
        lon, lat = coords[:2]
    elif gtype == "POLYGON":                   # [[[lon, lat], …]]
        lon, lat = coords[0][0][:2]            # first vertex is good enough
    else:                                      # unexpected but try MP-like
        lon, lat = coords[0][:2]               # works for MultiPoint
    return lat, lon

# ── HTTP helper ──────────────────────────────────────────────
async def _get_json(path: str, **params):
    async with httpx.AsyncClient(headers=_HEAD, timeout=10) as cx:
        r = await cx.get(f"{_FT_BASE}{path}", params=params)
    r.raise_for_status()
    return r.json()

# ── Public helpers ───────────────────────────────────────────
async def nearest_parking_ids(lat: float, lon: float, *, radius_m: int = 800) -> Dict[str, int]:
    rows = (await _get_json("/facilities", lat=lat, lon=lon, within__maxDistance=radius_m))["results"]
    here = (lat, lon)
    best: Dict[str, Tuple[float, int]] = {}
    for fac in rows:
        plat, plon = _latlon(fac)
        for ctype, cap in fac.get("builtCapacity", {}).items():
            if cap <= 0:
                continue
            dist = _haversine(here, (plat, plon))
            if ctype not in best or dist < best[ctype][0]:
                best[ctype] = (dist, fac["id"])
    return {k: v[1] for k, v in best.items()}

async def parking_status(facility_id: int, *, capacity_type="CAR", usage="PARK_AND_RIDE") -> dict:
    rows = await _get_json(f"/facilities/{facility_id}/utilization")
    row  = next(r for r in rows if r["capacityType"] == capacity_type and r["usage"] == usage)
    ts   = (dt.datetime.fromisoformat(row["timestamp"])
            .astimezone(dt.timezone.utc).isoformat(timespec="seconds"))
    return {
        "capacity":        row["capacity"],
        "spacesAvailable": row["spacesAvailable"],
        "timestamp_utc":   ts,
        "openNow":         row["openNow"],
    }

async def facility_prediction(facility_id: int, *, hours: int = 24) -> List[dict]:
    return await _get_json(f"/facilities/{facility_id}/prediction", after=f"{hours}:00")
